In [6]:
import openai
import time
import numpy as np
import pandas as pd
import math

In [ ]:
home_wins_5 = 4
home_loss_5 = 1
away_win_5 = 2
away_loss_5 = 3
h2h_home_goals_5 = [0,2,3,1,1]
h2h_away_goals_5 = [2,2,1,1,0]
home_tean = "Manchester Utd"
away_teamm = "machester City"
home_win_prob = 60
away_win_prob = 20
draw_prob = 20

### Poisson formula: P(x, λ ) =(e– λ λx)/x!

### EG adjusted = average goals as home/awayH2H * average GF as home last 5 games/ average GA as away last 5 games

In [34]:
home_eg = 0.99
away_eg = 3
def poisson_goal(g,eg):
    return (math.e**(-eg)*eg**g)/(math.factorial(g))
home_goals_probs = []
for i in range(0,8):
    prob = poisson_goal(i,home_eg)
    home_goals_probs.append(prob)
    

In [35]:
away_goals_probs = []
for i in range(0,8):
    prob = poisson_goal(i,away_eg)
    away_goals_probs.append(prob)

In [53]:
away_goals_probs

[0.04978706836786395,
 0.14936120510359185,
 0.22404180765538778,
 0.22404180765538775,
 0.16803135574154082,
 0.10081881344492451,
 0.05040940672246225,
 0.02160403145248382]

In [37]:
home_goals_probs

[0.3715766910220457,
 0.36786092411182525,
 0.1820911574353535,
 0.060090081953666656,
 0.014872295283532497,
 0.002944714466139434,
 0.0004858778869130067,
 6.871701543483951e-05]

In [42]:
draw_prob = sum(np.array(home_goals_probs)*np.array(away_goals_probs))

In [47]:
home_win_prob_list = []
for i in range(0,len(home_goals_probs)):
    for j in range(0,len(away_goals_probs)):
        if j < i:
            home_win_prob_list.append(home_goals_probs[i]*away_goals_probs[j])
            

In [49]:
home_win_prob = sum(home_win_prob_list)

0.09254544291339192

In [51]:
away_win_prob = 1 - draw_prob - home_win_prob

In [52]:
away_win_prob

0.7769300974395149

In [39]:
prompt = """Bạn là một chuyên gia về nhận định kèo đấu bóng đá. Bạn đưa ra nhận định dựa trên những số liệu được cung cấp
mà không sử dụng thêm thông tin bên ngoài.  Các số liệu thống kê trước trận đấu giữa team A và team B được trình bày như sau:
"Trong 5 lần gặp nhau gần nhất giữa team A và team B, team A thắng 3 thua 1 và hòa 1. Trong 5 trận gần nhất của giải đấu,
team A thắng 1, thua 4 hòa 0. Trong 5 trận gần nhất của giải đấu team B thắng 3, hòa 2, thua 0.
Các chuyên gia dự đoán tỉ số của trận đấu này sẽ là 1-1. Xác suất thắng của team A là 60%, xác suất thắng của team B là 10% 
và xác suất để 2 đội hòa nhau là 30% ". Hãy viết một bài nhận định và dự đoán về trận đấu giữa team A và team B.
Bài viết phải có sắc thái chuyên nghiệp, khách quan và có tính thuyết phục. 
Bài viết phải sử dụng toàn bộ các thông tin đã được cung cấp."""

In [40]:
type(prompt)

str